In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("Training Data.csv").drop('Id',axis=1)
test_df  = pd.read_csv("Test Data.csv").drop('id',axis=1)
test_df['risk_flag'] = 0

train_data = pd.concat([train_df,test_df],axis=0)

In [3]:
def get_state_dict():
    # This encoding is done based on per-capita-income-rating

    state_dict ={'Sikkim': 0,
                 'Delhi': 1,
                 'Chandigarh': 2,
                 'Haryana': 3,
                 'Karnataka': 4,
                 'Puducherry': 5,
                 'Kerala': 6,
                 'Telangana': 7,
                 'Gujarat': 8,
                 'Uttarakhand': 9,
                 'Maharashtra': 10,
                 'Tamil Nadu': 11,
                 'Himachal Pradesh': 12,
                 'Mizoram': 13,
                 'Punjab': 14,
                 'Andhra Pradesh': 15,
                 'Tripura': 16,
                 'Rajasthan': 17,
                 'Odisha': 18,
                 'Jammu and Kashmir': 19,
                 'West Bengal': 20,
                 'Chhattisgarh': 21,
                 'Madhya Pradesh': 22,
                 'Assam': 23,
                 'Manipur': 24,
                 'Jharkhand': 25,
                 'Uttar Pradesh': 26,
                 'Bihar': 27}
    return state_dict


def get_cities_dict():
    # Ranking goes top to Bottom

    cities_list_ranked = ["Bengaluru","Pune","Ahmedabad","Chennai","Surat","Navi Mumbai","Coimbatore","Vadodara","Indore",
          "Greater Mumbai","Thane","Kalyan Dombivali","New Delhi","Noida","Ludhiana","Visakhapatnam","Pimpri Chinchwad",
    "Solapur","Raipur","Bhopal","Rajkot","Jodhpur","Madurai","Jaipur","Hyderabad","Nagpur","Lucknow","Varanasi",
    "Kanpur","Chandigarh","Ghaziabad","Gwalior","Prayagraj","Patna","Aurangabad","Agra","Meerut","Hubli Dharwad",
    "Nashik","Vasai Virar","Faridabad","Vijayawada","Ranchi","Jabalpur","Kota","Amritsar","Guwahati","Barielly",
    "Dhanbad","Srinagar" ]

    # The cities which are not present will be added at last, and one feature would be if they are present or not

    bool_city = list(np.ones((len(cities_list_ranked)),dtype = np.int64))
    
    for i in train_data.city.values:
        i = i.split("[")[0]
        if i not in cities_list_ranked:
            bool_city.append(0)
            cities_list_ranked.append(i)

    # Create a dictionary for Mapping

    cities_dict = {}
    for i,city in enumerate(cities_list_ranked):
        cities_dict[city] = i

    return cities_dict

def get_profess_dict(method):
    
    if method == 1 :
        # Based of Avg Salary
        
        profess_mean = []

        for i in train_data.profession.unique():
            inc = train_data[train_data.profession == i]["income"].mean()
            profess_mean.append([i,inc])

        profess_mean = np.array(profess_mean)[np.array(profess_mean)[:,1].argsort()]

        profess_dict = {}

        for i,prof in enumerate(profess_mean):
            profess_dict[prof[0]] = i
        
        return profess_dict
    
    elif method == 2 :
        # Based on risk flag

        profess_mean = []

        for i in train_data.profession.unique():
            inc = train_data[train_data.profession == i]["risk_flag"].sum()
            profess_mean.append([i,inc])

        profess_mean = np.array(profess_mean)[np.array(profess_mean)[:,1].argsort()]

        profess_dict = {}

        for i,prof in enumerate(profess_mean):
            profess_dict[prof[0]] = i

        return profess_dict

In [4]:
# Process profession --- >

profess = train_data.profession.unique()
new_profess = []

for i in profess:
    i = i.replace("_", " ")
    new_profess.append(i)
    
for i, j in zip(profess,new_profess):
    train_data.profession = train_data.profession.replace(i,j)
    

    
for j,i in enumerate(train_data.city.values):
    train_data.city.values[j] = i.split("[")[0]
    

    
    
# Process state  --- >

state = train_data.state.unique()
new_state = []

for i in state:
    i = i.replace("_", " ")
    new_state.append(i)
    
for i, j in zip(state,new_state):
    train_data.state = train_data.state.replace(i,j)
    

train_data.state = train_data.state.replace("Uttar Pradesh[5]","Uttar Pradesh")

In [5]:
train_data

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1303835,23,3,single,rented,no,Mechanical engineer,Rewa,Madhya Pradesh,3,13,0
1,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
2,3991815,66,4,married,rented,no,Technical writer,Alappuzha,Kerala,4,10,0
3,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
4,5768871,47,11,single,rented,no,Civil servant,Tiruchirappalli,Tamil Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...
27995,9955481,57,13,single,rented,no,Statistician,Eluru,Andhra Pradesh,5,10,0
27996,2917765,47,9,single,rented,no,Technical writer,Ratlam,Madhya Pradesh,9,14,0
27997,8082415,24,5,single,rented,no,Lawyer,Mira-Bhayandar,Maharashtra,4,13,0
27998,9474180,51,13,single,rented,yes,Chartered Accountant,Bhilai,Chhattisgarh,13,14,0


In [6]:
profession_sort = 2 # 1 or 2

train_data.state = train_data.state.map(get_state_dict()).astype(np.int64)
train_data.city = train_data.city.map(get_cities_dict())
train_data.profession = train_data.profession.map(get_profess_dict(profession_sort))


train_data["married"]=pd.factorize(train_data.married)[0]
train_data["house_ownership"]=pd.factorize(train_data.house_ownership)[0]
train_data["car_ownership"]=pd.factorize(train_data.car_ownership)[0]

In [8]:
train_out = train_data[:len(train_df)]
test_out  = train_data[len(train_df):]

In [11]:
train_out.to_csv("train_encoded.csv",index=False)
test_out.to_csv("test_encoded.csv",index=False)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
concat